# Xilium IA

> FA2 | BARKER, OUALI, GUILLERAY, GRAVIER, LEMOUTON  

## Introduction

Xilium est une application de ticketing qui permet au personnel de créer des tickets si ils ont des problèmes informatiques que des techniciens pourront résoudre si besoin. Le membre du personnel va donc attribuer une catégorie à son ticket pour qu'il soit plus facilement triable et traitable. Cependant les techniciens et les administrateurs peuvent adapter le champ de catégorie en fonction de leur préférence pour qu'il corresponde reelement au problème posé. Pour cela, nous avons créé une IA qui va determiner a partir de la description du ticket sa catégorie et en avoir une potentiellement plus pertinente. Si la categorie determiné par l'IA est la meme que celle mis par le membre du personnel alors rien ne change. Si la categorie proposé est différente, le technicien ou l'administrateur peuvent valider la suggestion de l'IA et avoir une categorie pertinente pour le ticket.
 

## Le programme
### Importation des librairies


In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

### Lecture du csv et creation des sets

In [2]:
df = pd.read_csv("./src/app/ai/data/tickets.csv")
X = df.description
y = df.categorie
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [3]:
df

,type,description,priorite,categorie
0,Demande,Impossible d'ouvrir le projet dans l'IDE,Elevé,Logiciel
1,Incident,Le serveur de développement est inaccessible,Moyenne,Réseau
2,Demande,Nouvelle fonctionnalité pour le système de ges...,Faible,Logiciel
3,Incident,Le matériel de développement est défectueux,Elevé,Matériel
4,Demande,Ajout d'un nouveau chapitre dans la documentation,Faible,Documentation
...,...,...,...,...
547,Demande,Ajouter des ressources pour les étudiants étra...,Moyenne,Documentation
548,Incident,La documentation de Apache Flink est introuvable,Élevée,Documentation
549,Demande,Créer un guide de démarrage rapide pour les no...,Basse,Documentation
550,Incident,La documentation de Apache Spark est erronée,Haute,Documentation


In [4]:
df.categorie.value_counts()

categorie
Logiciel         138
Réseau           138
Matériel         138
Documentation    138
Name: count, dtype: int64

### Création du pipeline et du GridSearch pour fit les données puis les prédire

In [5]:
pip = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("model", LogisticRegression(class_weight='balanced', max_iter=1000)) # On fait une regression logistique équlibré avec nos données
])
parameters = {"tfidf__ngram_range": [(1, 1), (1, 2)],
              "model__C": np.logspace(-2, 2, 5)}
gs = GridSearchCV(pip, parameters, verbose=1)
gs.fit(X_train, y_train) # On fit les données, c'est a dire qu'on creer le modèle pour pouvoir prédire des données
y_test_pred = gs.predict(X_test) # On prédit les données de test

Fitting 5 folds for each of 10 candidates, totalling 50 fits


### Rapport de classification 

todo

In [6]:
print(classification_report(y_test, y_test_pred))

               precision    recall  f1-score   support

Documentation       1.00      1.00      1.00        43
     Logiciel       1.00      1.00      1.00        35
     Matériel       0.96      1.00      0.98        26
       Réseau       1.00      0.97      0.99        34

     accuracy                           0.99       138
    macro avg       0.99      0.99      0.99       138
 weighted avg       0.99      0.99      0.99       138

